In [1]:
# Interpret Q&As. Chinese word segmentation, parsing, prepare for machine learning

import os
import re
import pandas as pd
import subprocess
import jieba
import jieba.analyse

In [62]:
# combine doctors to a single dataframe and generate summary
folder = sorted([x for x in os.listdir('../data') if \
                 x.startswith('data') and not x.endswith('.csv')])[-1]
data = pd.DataFrame()
stat = list()
for i in [x for x in os.listdir('../data/' + folder) if x.endswith('.csv')]:
    csv = '../data/' + folder +'/'+ i
    df = pd.read_csv(csv, index_col=0,encoding='utf-8', error_bad_lines=False)
    log = csv[:-3]+'log'
    bashCommand = 'tail -2 ' + log
    out = subprocess.check_output(bashCommand.split()).decode("utf-8")
    valid = int(re.search('[0-9]+ Q&As', out).group(0).split()[0])
    total = int(re.search('[0-9]+ posts', out).group(0).split()[0])
    stat.append({'index':i[:-4],'医生姓名':df.ix[0]['大夫'],'有效问答':valid,'总问答':total})
    data = data.append(df, ignore_index=True)

summary = pd.DataFrame(stat, columns=['index','医生姓名','有效问答','总问答'])
sum_row = summary[['有效问答','总问答']].sum()
sum_row['index'] = '总计'
sum_row['医生姓名'] = '共'+str(len(summary.index))+'人'
summary = summary.append(pd.DataFrame(data=sum_row).T, ignore_index=True)
summary['有效率'] = summary['有效问答']/summary['总问答']
summary.to_csv('../data/' + folder +'/summary.csv', encoding='GBK')
data.to_csv('../data/' + folder + '.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc9 in position 2: invalid continuation byte

In [40]:
# read in latest data
datafile = sorted([x for x in os.listdir('../data') if \
                   x.startswith('data') and x.endswith('.csv')])[-1] # read in latest data
df = pd.read_csv('../data/' + datafile, encoding='utf-8', index_col=0)
print(datafile, 'read in.')

data-2016-09-27.csv read in.


In [47]:
def beforeAfter(s):
    seg_list = jieba.cut(s)
    print('\nBefore:\n' + s)
    print('\nAfter:\n' + '/'.join(seg_list))
    

In [53]:
len(df.index)

42945

In [58]:
test = df.ix[42943]['病情描述']
beforeAfter(test)

TypeError: Can't convert 'float' object to str implicitly

In [24]:
jieba.analyse.extract_tags(test, withWeight=True)

[('哭闹', 0.24611404264772585),
 ('爷爷', 0.22834251369595016),
 ('妈妈', 0.18379381714654208),
 ('大人', 0.11764099291140187),
 ('母亲', 0.10888028065981306),
 ('母乳', 0.09695623483551402),
 ('依赖', 0.08678298960922119),
 ('看见', 0.07921039954859813),
 ('下班', 0.07805235109420561),
 ('二宝', 0.0744845327283489),
 ('动画片', 0.06262012492398754),
 ('同龄', 0.062488951034890966),
 ('大宝', 0.06211122753426792),
 ('吃饭', 0.06022638408224299),
 ('父母', 0.059701496329906546),
 ('离家', 0.054833714792211836),
 ('经常', 0.05445468631635514),
 ('行为', 0.05034101840728972),
 ('之前', 0.04853762127700935),
 ('孩子', 0.048423927817476634)]

In [25]:
jieba.analyse.textrank(test, withWeight=True)

[('爷爷', 1.0),
 ('妈妈', 0.9021897662991276),
 ('哭闹', 0.8919115792066067),
 ('大人', 0.8556454183936896),
 ('母亲', 0.8112763781262181),
 ('看见', 0.7326841665119904),
 ('吃饭', 0.5970104275626192),
 ('开始', 0.5630963372934767),
 ('行为', 0.5583041672657686),
 ('社会', 0.5133030986530981),
 ('不会', 0.4588479378070064),
 ('依赖', 0.44445791430613973),
 ('下班', 0.4112026106406721),
 ('离家', 0.41040473886223205),
 ('没有', 0.4089467922073848),
 ('能够', 0.40697253672681766),
 ('父母', 0.40538612154431924),
 ('喜欢', 0.3994532787848826),
 ('给予', 0.39298663149871993),
 ('不见', 0.3796648082779753)]